In [1]:
from pathlib import Path
pred_path = Path('/hdd/yang/results/glomeruli_segmentation/nnUNet_results/Dataset015_Care2025_benchmark3/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_2/train_all/')
pred_files = list(pred_path.glob('*.nii.gz'))
pred_files.sort()
images_path = Path('/hdd/yang/data/care2025/validation_dataset/train_all')
images = list(images_path.glob('*.nii.gz'))
images.sort()
# moving
pred_des_path = '/hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset016_Care2025_benchmark4/labelsTr'
image_des_path = '/hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset016_Care2025_benchmark4/imagesTr'
for pred_file in pred_files:
    image_name = pred_file.name.split('.')[0] + '_0000.nii.gz'
    try:
        pred_file.rename(Path(pred_des_path) / pred_file.name)
        (images_path / image_name).rename(Path(image_des_path) / image_name)
    except FileNotFoundError:
        print(f"File not found: {image_name}, skipping {pred_file.name}")
print(f"Moved {len(pred_files)} prediction files and corresponding images to the new dataset structure.")


Moved 2464 prediction files and corresponding images to the new dataset structure.


In [61]:
import nibabel as nib
import numpy as np
import skimage.exposure as skex

def normalise_image(im_path):
#This function normalises the image slice by slice.
    '''
    Args:
        im_path: str, path to the image
    Returns:
        norm_slice: np.array, normalised slice
    '''
    image = nib.load(im_path)
    #print(image)
    data_type = image.get_data_dtype()
    print(f'Normalising image {im_path} of type {data_type}.')
    
    image_data = image.get_fdata()

    # normalise by max and min
    new_image = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))
    new_image = (new_image * 255).astype(np.uint8)
    # change image header to reflect the new data type
    image.header.set_data_dtype(np.uint8)
    # create new image with the same affine and header
    norm_data = nib.Nifti1Image(new_image, image.affine, image.header)

    return norm_data

In [56]:
import nibabel as nib
sample_data = '/hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset017_Care2025_benchmark5/imagesTr/1998-A-S4_GED4_350_0000.nii.gz'
norm_image = normalise_image(sample_data)
print(norm_image)


<class 'nibabel.nifti1.Nifti1Image'>
data shape (640, 520, 64)
affine:
[[  -0.59380001    0.            0.          176.7991333 ]
 [   0.           -0.59380001    0.          134.70124817]
 [   0.            0.            3.5        -136.35241699]
 [   0.            0.            0.            1.        ]]
metadata:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 640 520  64   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : int16
bitpix          : 16
slice_start     : 0
pixdim          : [1.     0.5938 0.5938 3.5    0.     0.     0.     0.    ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 2
cal_max         : 0.0
cal_min         : 0.0
slice_duration  :

In [57]:
from ipywidgets import fixed, interact
import matplotlib.pyplot as plt
%matplotlib inline

image_arr = norm_image.get_fdata()
#image_arr = image_arr.transpose(2, 1, 0)  # Transpose to (depth, height, width)
print(f"Image shape: {image_arr.shape}")
print(image_arr.max(), image_arr.min())

def display_slice(slice_index):
    slice = image_arr[:, :, slice_index]  # Extract the specified slice
    slice = np.asarray(slice, dtype=np.uint8)  # Ensure the slice is in uint8 format
    plt.imshow(slice, cmap='gray')
    plt.title(f'Slice {slice_index}')
    plt.axis('off')
    plt.show()

interact(
    display_slice, 
    slice_index=(0, image_arr.shape[2] - 1, 1)
    )


Image shape: (640, 520, 64)
255.0 0.0


interactive(children=(IntSlider(value=31, description='slice_index', max=63), Output()), _dom_classes=('widget…

<function __main__.display_slice(slice_index)>

In [58]:
image = nib.load(sample_data)
original_data = image.get_fdata()
# Display the original image data
def display_slice(slice_index):/hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/
    plt.show()
interact(
    display_slice, 
    slice_index=(0, image_arr.shape[2] - 1, 1)
    )

interactive(children=(IntSlider(value=31, description='slice_index', max=63), Output()), _dom_classes=('widget…

<function __main__.display_slice(slice_index)>

In [63]:
from pathlib import Path
im_dir = Path('/hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/')
norm_dir = Path('/hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset017_Care2025_benchmark5/imagesTr/')
if not norm_dir.exists():
    norm_dir.mkdir(parents=True, exist_ok=True)
for im_file in im_dir.glob('*.nii.gz'):
    norm_image = normalise_image(im_file)
    norm_image.to_filename(norm_dir / im_file.name)


Normalising image /hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/0487-B1-S2_1144-B1-S1_img_027_0000.nii.gz of type float64.
Normalising image /hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/1432-A-S4_1406-A-S1_img_270_0000.nii.gz of type float64.
Normalising image /hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/1920-A-S1_GED3_289_0000.nii.gz of type int16.
Normalising image /hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/1070-B2-S4_GED4_153_0000.nii.gz of type float32.
Normalising image /hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/1098-B2-S4_T2_210_0000.nii.gz of type float32.
Normalising image /hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_Care2025_benchmark3/imagesTr/1991-A-S4_1927-A-S1_img_331_0000.nii.gz of type float64.
Normalising image /hdd/yang/data/kidney_seg_nnunet/nnUNet_raw/Dataset015_